In [20]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


from library.sb_utils import save_file

%matplotlib inline

In [2]:
path_train = r'C:\Users\adame\OneDrive\Desktop\python_scripts\data\titanic\train.csv'
df_train = pd.read_csv(path_train)

path_test = r'C:\Users\adame\OneDrive\Desktop\python_scripts\data\titanic\test.csv'
df_test = pd.read_csv(path_test)

Check train data

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Check test data

In [5]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [7]:
def check_data(dataframe, head=5):
    print(" SHAPE ".center(70,'-'))
    print('Rows: {}'.format(dataframe.shape[0]))
    print('Columns: {}'.format(dataframe.shape[1]))
    print(" TYPES ".center(70,'-'))
    print(dataframe.dtypes)
    print(" HEAD ".center(70,'-'))
    print(dataframe.head(head))
    print(" TAIL ".center(70,'-'))
    print(dataframe.tail(head))
    print(" MISSING VALUES ".center(70,'-'))
    print(dataframe.isnull().sum())
    print(" DUPLICATED VALUES ".center(70,'-'))
    print(dataframe.duplicated().sum())
    print(" QUANTILES ".center(70,'-'))
    # print the quantiles in the data, [0, 0.05, 0.50, 0.95, 0.99, 1]
    print(dataframe.select_dtypes(include=np.number).quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
    
check_data(df_train)

------------------------------- SHAPE --------------------------------
Rows: 891
Columns: 12
------------------------------- TYPES --------------------------------
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
-------------------------------- HEAD --------------------------------
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                          

Explore the data

In [8]:
# the first hypothesis is that all women survived, 
# this code returns the female survivor rate to test that hypothesis

women = df_train.loc[df_train.Sex == 'female']['Survived']
rate_women = round(sum(women) / len(women), 2) * 100

print('% of women who survived: ', rate_women)

% of women who survived:  74.0


In [9]:
# same as above, but for men
men = df_train.loc[df_train.Sex == 'male']['Survived']
rate_men = round(sum(men) / len(men), 2) * 100

print('% of men who survived: ', rate_men)

% of men who survived:  19.0


Random Forest Model

In [10]:
# set target column to y values
y = df_train['Survived']

# make a list of features to dummy encode
features = ['Pclass', 'Sex', 'SibSp', 'Parch']

# encode categorical feature with dummy encoding
X = pd.get_dummies(df_train[features])

# define the test data
X_test = pd.get_dummies(df_test[features])

In [11]:
# instantiate the model
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=5)
# fit the model
model.fit(X, y)
# make predictions with the trained model
y_pred = model.predict(X_test)

In [12]:
# score the initial model


In [24]:
param_grid = {
    'n_estimators':[100,200,300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2,5],
    'min_samples_leaf': [1,2,3]
}

In [33]:
second_model = RandomForestClassifier(random_state=42)
second_model_GridSearchCV = GridSearchCV(estimator=second_model, param_grid=param_grid, n_jobs=-1, cv=5, scoring='accuracy')
second_model_GridSearchCV.fit(X, y)
second_model_predictions = second_model_GridSearchCV.predict(X_test)

In [34]:
best_rf = second_model_GridSearchCV.best_estimator_

print('Best parameters found: ', second_model_GridSearchCV.best_params_)

Best parameters found:  {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}


In [35]:
# score the second model
second_model_GridSearchCV.score(X, y)

0.8125701459034792

In [36]:
second_model_GridSearchCV.best_params_

{'max_depth': 5,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 300}

In [37]:
second_model_GridSearchCV.best_score_

0.8036218693114054

In [38]:
output = pd.DataFrame({'PassengerId':df_test.PassengerId, 'Survived': second_model_predictions})
output.to_csv('submission.csv', index=False)
print('Your submission was successfully saved!')

Your submission was successfully saved!
